In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, mean_absolute_error, f1_score, confusion_matrix, precision_score, roc_curve, auc
from scipy.signal import savgol_filter
from tqdm import tqdm
from PyEMD import EEMD
import time

plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
sns.set(style='whitegrid')

torch.manual_seed(42)
np.random.seed(42)


class Config:
    symbols = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'NVDA', 'TSLA']
    start_date = '2012-01-01'
    end_date = pd.Timestamp.now().strftime('%Y-%m-%d')
    seq_length = 20
    train_ratio = 0.8
    batch_size = 128
    epochs = 200
    lr = 5e-4
    weight_decay = 1e-4
    dropout = 0.2
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_config = {
        'Transformer': {'d_model': 128, 'nhead': 8, 'num_layers': 3},
        'Informer': {'d_model': 128, 'nhead': 8, 'num_layers': 3}
    }


class DataProcessor:
    def __init__(self):
        self.scaler = StandardScaler()

    def get_data(self, symbol):
        max_retries = 3
        for attempt in range(max_retries):
            try:
                df = yf.download(symbol, start=Config.start_date, end=Config.end_date)
                return df[['Open', 'High', 'Low', 'Close', 'Volume']].dropna()
            except yf.YFRateLimitError:
                print(f"Rate limit reached for {symbol}. Retrying in 60 seconds...")
                time.sleep(60)
        print(f"Failed to download data for {symbol} after {max_retries} attempts.")
        return None

    def compute_rsi(self, prices, period=14):
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
        rs = gain / loss
        return 100 - (100 / (1 + rs))

    def compute_macd(self, prices, fast=12, slow=26, signal=9):
        ema_fast = prices.ewm(span=fast, adjust=False).mean()
        ema_slow = prices.ewm(span=slow, adjust=False).mean()
        macd = ema_fast - ema_slow
        signal_line = macd.ewm(span=signal, adjust=False).mean()
        return macd, signal_line

    def add_features(self, df):
        df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
        df['MA5'] = df['Close'].rolling(5).mean()
        df['MA20'] = df['Close'].rolling(20).mean()
        df['RSI'] = self.compute_rsi(df['Close'])
        macd, signal = self.compute_macd(df['Close'])
        df['MACD'] = macd
        df['MACD_Signal'] = signal
        df['DayOfWeek'] = df.index.dayofweek
        df['Month'] = df.index.month - 1
        return df.dropna()

    def decompose_with_eemd(self, series):
        eemd = EEMD()
        T = np.arange(len(series))
        imfs = eemd.emd(series, T)
        return imfs

    def plot_eemd_decomposition(self, symbol, series, imfs):
        plt.figure(figsize=(12, 8))
        plt.subplot(len(imfs) + 1, 1, 1)
        plt.plot(series, label='series')
        plt.title(f'{symbol} EEMD')
        plt.legend()

        for i, imf in enumerate(imfs):
            plt.subplot(len(imfs) + 1, 1, i + 2)
            plt.plot(imf, label=f'IMF_{i}')
            plt.legend()

        plt.tight_layout()
        plt.savefig(f'{symbol}_eemd_decomposition.png')
        plt.close()

    def process_data(self, symbol, use_imf=True, use_tech_indicators=True):
        df = self.get_data(symbol)
        if df is None:
            return None, None
        df = self.add_features(df)
        window_length = min(11, len(df['LogReturn']))
        df['LogReturn'] = savgol_filter(df['LogReturn'], window_length, 2)

        log_return_series = df['LogReturn'].values
        imfs = self.decompose_with_eemd(log_return_series)
        self.plot_eemd_decomposition(symbol, log_return_series, imfs)

        if use_imf:
            for i, imf in enumerate(imfs):
                df[f'IMF_{i}'] = imf

        base_features = ['LogReturn', 'MA5', 'MA20', 'DayOfWeek', 'Month']
        tech_features = ['RSI', 'MACD', 'MACD_Signal']
        features = base_features.copy()
        if use_tech_indicators:
            features.extend(tech_features)
        if use_imf:
            for i in range(len(imfs)):
                features.append(f'IMF_{i}')
        target = 'LogReturn'
        df[features] = self.scaler.fit_transform(df[features])

        X, y = [], []
        for i in range(len(df) - Config.seq_length):
            X.append(df[features].values[i:i + Config.seq_length])
            y.append(df[target].values[i + Config.seq_length])
        return np.array(X), np.array(y)


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dropout, use_positional_encoding=True):
        super().__init__()
        self.encoder = nn.Linear(input_size, d_model)
        self.use_positional_encoding = use_positional_encoding
        if use_positional_encoding:
            self.pos_encoder = PositionalEncoding(d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.decoder = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.encoder(x)
        if self.use_positional_encoding:
            x = self.pos_encoder(x)
        memory = self.transformer_encoder(x)

        tgt = x
        if self.use_positional_encoding:
            tgt = self.pos_encoder(tgt)
        output = self.transformer_decoder(tgt, memory)

        output = self.norm(output.mean(dim=1))
        output = self.dropout(output)
        return self.decoder(output)


class ProbSparseAttention(nn.Module):
    def __init__(self, d_model, nhead, factor):
        super().__init__()
        self.attention = nn.MultiheadAttention(d_model, nhead, dropout=Config.dropout, batch_first=True)
        self.factor = factor

    def forward(self, q, k, v):
        attn_output, _ = self.attention(q, k, v)
        return attn_output


class InformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, dropout, factor=5, use_positional_encoding=True, use_probs_sparse_attn=True):
        super().__init__()
        self.encoder = nn.Linear(input_size, d_model)
        self.use_positional_encoding = use_positional_encoding
        if use_positional_encoding:
            self.pos_encoder = PositionalEncoding(d_model)
        self.use_probs_sparse_attn = use_probs_sparse_attn
        if use_probs_sparse_attn:
            self.attn_layers = nn.ModuleList([
                ProbSparseAttention(d_model, nhead, factor) for _ in range(num_layers)
            ])
        else:
            self.attn_layers = nn.ModuleList([
                nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True) for _ in range(num_layers)
            ])

        decoder_layer = nn.TransformerDecoderLayer(
            d_model, nhead, dim_feedforward=d_model * 4, dropout=dropout, batch_first=True
        )
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.decoder = nn.Linear(d_model, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.encoder(x)
        if self.use_positional_encoding:
            x = self.pos_encoder(x)
        for attn in self.attn_layers:
            if isinstance(attn, ProbSparseAttention):
                x = attn(x, x, x)
            else:
                # 确保输入是 torch.Tensor 类型
                x, _ = attn(x, x, x)
        memory = x

        tgt = x
        if self.use_positional_encoding:
            tgt = self.pos_encoder(tgt)
        output = self.transformer_decoder(tgt, memory)

        output = self.norm(output.mean(dim=1))
        output = self.dropout(output)
        return self.decoder(output)


class Trainer:
    def __init__(self, model, model_name):
        self.model = model.to(Config.device)
        self.model_name = model_name
        self.criterion = nn.HuberLoss(delta=1.0)
        self.optimizer = optim.AdamW(model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
        self.warmup_scheduler = optim.lr_scheduler.LinearLR(self.optimizer, start_factor=0.1, total_iters=10)
        self.cosine_scheduler = optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=Config.epochs - 10)
        self.best_loss = float('inf')
        self.early_stop_counter = 0

    def train_epoch(self, train_loader, epoch):
        self.model.train()
        total_loss = 0
        for X, y in train_loader:
            X, y = X.to(Config.device), y.to(Config.device)
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs.squeeze(), y)
            loss.backward()
            nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            total_loss += loss.item()
        if epoch < 10:
            self.warmup_scheduler.step()
        else:
            self.cosine_scheduler.step()
        return total_loss / len(train_loader)

    def evaluate(self, test_loader):
        self.model.eval()
        total_loss = 0
        preds, truths = [], []
        with torch.no_grad():
            for X, y in test_loader:
                X, y = X.to(Config.device), y.to(Config.device)
                outputs = self.model(X)
                loss = self.criterion(outputs.squeeze(), y)
                total_loss += loss.item()
                preds.extend(outputs.cpu().numpy().flatten())
                truths.extend(y.cpu().numpy().flatten())
        return total_loss / len(test_loader), np.array(preds), np.array(truths)

    def train(self, train_loader, test_loader):
        for epoch in range(Config.epochs):
            train_loss = self.train_epoch(train_loader, epoch)
            test_loss, preds, truths = self.evaluate(test_loader)
            if test_loss < self.best_loss:
                self.best_loss = test_loss
                torch.save(self.model.state_dict(), f'best_{self.model_name}.pth')
                self.early_stop_counter = 0
            else:
                self.early_stop_counter += 1
                if self.early_stop_counter >= 15:
                    print(f"Early stopping at epoch {epoch}")
                    break
            if epoch % 10 == 0:
                print(f"Epoch {epoch + 1}/{Config.epochs} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f}")
        self.model.load_state_dict(torch.load(f'best_{self.model_name}.pth'))
        return preds, truths


class Evaluator:
    @staticmethod
    def calculate_metrics(preds, truths):
        pred_dir = (preds > 0).astype(int)
        true_dir = (truths > 0).astype(int)
        fpr, tpr, thresholds = roc_curve(true_dir, preds)
        roc_auc = auc(fpr, tpr)

        metrics = {
            'RMSE': np.sqrt(mean_squared_error(truths, preds)),
            'MAE': mean_absolute_error(truths, preds),
            'Accuracy': accuracy_score(true_dir, pred_dir),
            'Recall': recall_score(true_dir, pred_dir),
            'Sharpe': np.mean(preds) / np.std(preds) * np.sqrt(252) if np.std(preds) != 0 else 0,
            'Correlation': np.corrcoef(truths, preds)[0, 1],
            'F1': f1_score(true_dir, pred_dir),
            'ConfusionMatrix': confusion_matrix(true_dir, pred_dir),
            'Precision': precision_score(true_dir, pred_dir),
            'AUC': roc_auc
        }
        return metrics


def main():
    processor = DataProcessor()
    results = {}
    for symbol in Config.symbols:
        print(f"\n=== Processing {symbol} ===")

        # 原始特征
        X, y = processor.process_data(symbol, use_imf=True, use_tech_indicators=True)
        if X is None or y is None:
            continue
        train_size = int(len(X) * Config.train_ratio)
        X_train, X_test = X[:train_size], X[train_size:]
        y_train, y_test = y[:train_size], y[train_size:]
        train_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        test_dataset = torch.utils.data.TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
        train_loader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=Config.batch_size)
        model_results = {}
        input_size = X_train.shape[2]

        # 原始Transformer模型
        transformer = TransformerModel(input_size, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=True)
        trans_trainer = Trainer(transformer, 'Transformer')
        trans_preds, trans_truths = trans_trainer.train(train_loader, test_loader)
        model_results['Transformer'] = Evaluator.calculate_metrics(trans_preds, trans_truths)

        # 移除位置编码的Transformer模型
        transformer_no_pe = TransformerModel(input_size, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=False)
        trans_no_pe_trainer = Trainer(transformer_no_pe, 'Transformer_no_pe')
        trans_no_pe_preds, trans_no_pe_truths = trans_no_pe_trainer.train(train_loader, test_loader)
        model_results['Transformer_no_pe'] = Evaluator.calculate_metrics(trans_no_pe_preds, trans_no_pe_truths)

        # 原始Informer模型
        informer = InformerModel(input_size, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=True, use_probs_sparse_attn=True)
        informer_trainer = Trainer(informer, 'Informer')
        informer_preds, informer_truths = informer_trainer.train(train_loader, test_loader)
        model_results['Informer'] = Evaluator.calculate_metrics(informer_preds, informer_truths)

        # 移除位置编码的Informer模型
        informer_no_pe = InformerModel(input_size, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=False, use_probs_sparse_attn=True)
        informer_no_pe_trainer = Trainer(informer_no_pe, 'Informer_no_pe')
        informer_no_pe_preds, informer_no_pe_truths = informer_no_pe_trainer.train(train_loader, test_loader)
        model_results['Informer_no_pe'] = Evaluator.calculate_metrics(informer_no_pe_preds, informer_no_pe_truths)

        # 移除ProbSparseAttention的Informer模型
        informer_no_probs = InformerModel(input_size, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=True, use_probs_sparse_attn=False)
        informer_no_probs_trainer = Trainer(informer_no_probs, 'Informer_no_probs')
        informer_no_probs_preds, informer_no_probs_truths = informer_no_probs_trainer.train(train_loader, test_loader)
        model_results['Informer_no_probs'] = Evaluator.calculate_metrics(informer_no_probs_preds, informer_no_probs_truths)

        # 移除IMF特征
        X_no_imf, y_no_imf = processor.process_data(symbol, use_imf=False, use_tech_indicators=True)
        train_size_no_imf = int(len(X_no_imf) * Config.train_ratio)
        X_train_no_imf, X_test_no_imf = X_no_imf[:train_size_no_imf], X_no_imf[train_size_no_imf:]
        y_train_no_imf, y_test_no_imf = y_no_imf[:train_size_no_imf], y_no_imf[train_size_no_imf:]
        train_dataset_no_imf = torch.utils.data.TensorDataset(torch.FloatTensor(X_train_no_imf), torch.FloatTensor(y_train_no_imf))
        test_dataset_no_imf = torch.utils.data.TensorDataset(torch.FloatTensor(X_test_no_imf), torch.FloatTensor(y_test_no_imf))
        train_loader_no_imf = DataLoader(train_dataset_no_imf, batch_size=Config.batch_size, shuffle=True)
        test_loader_no_imf = DataLoader(test_dataset_no_imf, batch_size=Config.batch_size)
        input_size_no_imf = X_train_no_imf.shape[2]

        transformer_no_imf = TransformerModel(input_size_no_imf, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=True)
        trans_no_imf_trainer = Trainer(transformer_no_imf, 'Transformer_no_imf')
        trans_no_imf_preds, trans_no_imf_truths = trans_no_imf_trainer.train(train_loader_no_imf, test_loader_no_imf)
        model_results['Transformer_no_imf'] = Evaluator.calculate_metrics(trans_no_imf_preds, trans_no_imf_truths)

        informer_no_imf = InformerModel(input_size_no_imf, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=True, use_probs_sparse_attn=True)
        informer_no_imf_trainer = Trainer(informer_no_imf, 'Informer_no_imf')
        informer_no_imf_preds, informer_no_imf_truths = informer_no_imf_trainer.train(train_loader_no_imf, test_loader_no_imf)
        model_results['Informer_no_imf'] = Evaluator.calculate_metrics(informer_no_imf_preds, informer_no_imf_truths)

        # 移除技术指标特征
        X_no_tech, y_no_tech = processor.process_data(symbol, use_imf=True, use_tech_indicators=False)
        train_size_no_tech = int(len(X_no_tech) * Config.train_ratio)
        X_train_no_tech, X_test_no_tech = X_no_tech[:train_size_no_tech], X_no_tech[train_size_no_tech:]
        y_train_no_tech, y_test_no_tech = y_no_tech[:train_size_no_tech], y_no_tech[train_size_no_tech:]
        train_dataset_no_tech = torch.utils.data.TensorDataset(torch.FloatTensor(X_train_no_tech), torch.FloatTensor(y_train_no_tech))
        test_dataset_no_tech = torch.utils.data.TensorDataset(torch.FloatTensor(X_test_no_tech), torch.FloatTensor(y_test_no_tech))
        train_loader_no_tech = DataLoader(train_dataset_no_tech, batch_size=Config.batch_size, shuffle=True)
        test_loader_no_tech = DataLoader(test_dataset_no_tech, batch_size=Config.batch_size)
        input_size_no_tech = X_train_no_tech.shape[2]

        transformer_no_tech = TransformerModel(input_size_no_tech, **Config.model_config['Transformer'], dropout=Config.dropout, use_positional_encoding=True)
        trans_no_tech_trainer = Trainer(transformer_no_tech, 'Transformer_no_tech')
        trans_no_tech_preds, trans_no_tech_truths = trans_no_tech_trainer.train(train_loader_no_tech, test_loader_no_tech)
        model_results['Transformer_no_tech'] = Evaluator.calculate_metrics(trans_no_tech_preds, trans_no_tech_truths)

        informer_no_tech = InformerModel(input_size_no_tech, **Config.model_config['Informer'], dropout=Config.dropout, use_positional_encoding=True, use_probs_sparse_attn=True)
        informer_no_tech_trainer = Trainer(informer_no_tech, 'Informer_no_tech')
        informer_no_tech_preds, informer_no_tech_truths = informer_no_tech_trainer.train(train_loader_no_tech, test_loader_no_tech)
        model_results['Informer_no_tech'] = Evaluator.calculate_metrics(informer_no_tech_preds, informer_no_tech_truths)

        results[symbol] = model_results

        for model_name, preds in [
            ('Transformer', trans_preds), ('Transformer_no_pe', trans_no_pe_preds),
            ('Informer', informer_preds), ('Informer_no_pe', informer_no_pe_preds),
            ('Informer_no_probs', informer_no_probs_preds), ('Transformer_no_imf', trans_no_imf_preds),
            ('Informer_no_imf', informer_no_imf_preds), ('Transformer_no_tech', trans_no_tech_preds),
            ('Informer_no_tech', informer_no_tech_preds)
        ]:
            # 绘制预测结果
            plt.figure(figsize=(12, 6))
            plt.plot(trans_truths, label='True Returns', alpha=0.7)
            plt.plot(preds, label=f'{model_name} Predicted Returns', linestyle='--')
            plt.title(f'{symbol} Return Prediction - {model_name}')
            plt.legend()
            plt.savefig(f'{symbol}_prediction_ending1_{model_name}.png')
            plt.close()

            # 绘制ROC曲线
            pred_dir = (preds > 0).astype(int)
            true_dir = (trans_truths > 0).astype(int)
            fpr, tpr, thresholds = roc_curve(true_dir, preds)
            roc_auc = auc(fpr, tpr)
            plt.figure(figsize=(8, 8))
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'{symbol} {model_name} ROC Curve')
            plt.legend(loc="lower right")
            plt.savefig(f'{symbol}_roc_curve_{model_name}.png')
            plt.close()

    print("\n=== Final Results ===")
    for symbol in results:
        print(f"\n{symbol} Performance:")
        for model in results[symbol]:
            metrics = results[symbol][model]
            print(f"{model}:")
            print(f"  RMSE: {metrics['RMSE']:.4f} | MAE: {metrics['MAE']:.4f}")
            print(f"  Accuracy: {metrics['Accuracy']:.2%} | Recall: {metrics['Recall']:.2%}")
            print(f"  Sharpe: {metrics['Sharpe']:.2f} | Corr: {metrics['Correlation']:.2f}")
            print(f"  Precision: {metrics['Precision']:.2f}")
            print(f"  F1: {metrics['F1']:.2f}")
            print(f"  AUC: {metrics['AUC']:.2f}")
            print(f"  Confusion Matrix:\n{metrics['ConfusionMatrix']}")


if __name__ == '__main__':
    main()
    


=== Processing AAPL ===


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.4237 | Test Loss: 0.5543
Epoch 11/200 | Train Loss: 0.0680 | Test Loss: 0.2099
Epoch 21/200 | Train Loss: 0.0404 | Test Loss: 0.1646
Epoch 31/200 | Train Loss: 0.0336 | Test Loss: 0.3269
Early stopping at epoch 35
Epoch 1/200 | Train Loss: 0.4508 | Test Loss: 0.7586
Epoch 11/200 | Train Loss: 0.2511 | Test Loss: 0.5114
Epoch 21/200 | Train Loss: 0.1231 | Test Loss: 0.5274
Early stopping at epoch 30
Epoch 1/200 | Train Loss: 0.4785 | Test Loss: 0.5616
Epoch 11/200 | Train Loss: 0.0662 | Test Loss: 0.1591
Epoch 21/200 | Train Loss: 0.0496 | Test Loss: 0.1564
Epoch 31/200 | Train Loss: 0.0444 | Test Loss: 0.1728
Epoch 41/200 | Train Loss: 0.0423 | Test Loss: 0.2241
Early stopping at epoch 47
Epoch 1/200 | Train Loss: 0.4084 | Test Loss: 0.5672
Epoch 11/200 | Train Loss: 0.2911 | Test Loss: 0.6030
Epoch 21/200 | Train Loss: 0.2064 | Test Loss: 0.6168
Early stopping at epoch 28
Epoch 1/200 | Train Loss: 0.4270 | Test Loss: 0.5575
Epoch 11/200 | Train Loss: 0.0777

[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.4180 | Test Loss: 0.5529
Epoch 11/200 | Train Loss: 0.1522 | Test Loss: 0.2263
Epoch 21/200 | Train Loss: 0.1114 | Test Loss: 0.1984
Epoch 31/200 | Train Loss: 0.0917 | Test Loss: 0.2552
Early stopping at epoch 37
Epoch 1/200 | Train Loss: 0.4446 | Test Loss: 0.5566
Epoch 11/200 | Train Loss: 0.1592 | Test Loss: 0.2580
Epoch 21/200 | Train Loss: 0.1178 | Test Loss: 0.2261
Early stopping at epoch 29


...

Epoch 1/200 | Train Loss: 0.3820 | Test Loss: 0.4666
Epoch 11/200 | Train Loss: 0.0853 | Test Loss: 0.1640
Epoch 21/200 | Train Loss: 0.0393 | Test Loss: 0.1390
Epoch 31/200 | Train Loss: 0.0308 | Test Loss: 0.1394
Epoch 41/200 | Train Loss: 0.0226 | Test Loss: 0.1202
Epoch 51/200 | Train Loss: 0.0203 | Test Loss: 0.1230
Early stopping at epoch 53
Epoch 1/200 | Train Loss: 0.3866 | Test Loss: 0.4910
Epoch 11/200 | Train Loss: 0.2434 | Test Loss: 0.4814
Epoch 21/200 | Train Loss: 0.1335 | Test Loss: 0.6028
Epoch 31/200 | Train Loss: 0.0795 | Test Loss: 0.5028
Early stopping at epoch 38
Epoch 1/200 | Train Loss: 0.4119 | Test Loss: 0.5028
Epoch 11/200 | Train Loss: 0.0700 | Test Loss: 0.1612
Epoch 21/200 | Train Loss: 0.0515 | Test Loss: 0.1414
Epoch 31/200 | Train Loss: 0.0366 | Test Loss: 0.1448
Early stopping at epoch 39
Epoch 1/200 | Train Loss: 0.3842 | Test Loss: 0.4522
Epoch 11/200 | Train Loss: 0.2928 | Test Loss: 0.4774
Early stopping at epoch 15
Epoch 1/200 | Train Loss: 0.4284

[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.4152 | Test Loss: 0.5113
Epoch 11/200 | Train Loss: 0.1184 | Test Loss: 0.1630
Epoch 21/200 | Train Loss: 0.0892 | Test Loss: 0.1620
Epoch 31/200 | Train Loss: 0.0781 | Test Loss: 0.1788
Early stopping at epoch 35
Epoch 1/200 | Train Loss: 0.4671 | Test Loss: 0.5135
Epoch 11/200 | Train Loss: 0.1274 | Test Loss: 0.1881
Epoch 21/200 | Train Loss: 0.1027 | Test Loss: 0.1826
Epoch 31/200 | Train Loss: 0.0943 | Test Loss: 0.2051
Early stopping at epoch 37


[*********************100%***********************]  1 of 1 completed


Epoch 1/200 | Train Loss: 0.4144 | Test Loss: 0.5413
Epoch 11/200 | Train Loss: 0.0653 | Test Loss: 0.1495
Epoch 21/200 | Train Loss: 0.0425 | Test Loss: 0.1385
Epoch 31/200 | Train Loss: 0.0277 | Test Loss: 0.1031
Epoch 41/200 | Train Loss: 0.0225 | Test Loss: 0.1151
Early stopping at epoch 49
Epoch 1/200 | Train Loss: 0.3943 | Test Loss: 0.5049
Epoch 11/200 | Train Loss: 0.0623 | Test Loss: 0.1345
Epoch 21/200 | Train Loss: 0.0470 | Test Loss: 0.1499
Early stopping at epoch 29

=== Final Results ===

AAPL Performance:
Transformer:
  RMSE: 0.5499 | MAE: 0.3245
  Accuracy: 88.07% | Recall: 79.65%
  Sharpe: -3.96 | Corr: 0.88
  Precision: 0.97
  F1: 0.87
  AUC: 0.96
  Confusion Matrix:
[[309   9]
 [ 70 274]]
Transformer_no_pe:
  RMSE: 0.9128 | MAE: 0.6650
  Accuracy: 69.18% | Recall: 64.24%
  Sharpe: -2.19 | Corr: 0.49
  Precision: 0.73
  F1: 0.68
  AUC: 0.76
  Confusion Matrix:
[[237  81]
 [123 221]]
Informer:
  RMSE: 0.4646 | MAE: 0.3171
  Accuracy: 86.25% | Recall: 75.87%
  Sharpe: -

In [9]:
import csv

# Data for each stock
stocks_data = {
    "AAPL": [
        {"Model": "Transformer", "RMSE": 0.5499, "MAE": 0.3245, "Accuracy (%)": 88.07, "Recall (%)": 79.65, "Sharpe": -3.96, "Corr": 0.88, "Precision": 0.97, "F1": 0.87, "AUC": 0.96, "Confusion Matrix": "[[309, 9], [70, 274]]"},
        {"Model": "Transformer_no_pe", "RMSE": 0.9128, "MAE": 0.6650, "Accuracy (%)": 69.18, "Recall (%)": 64.24, "Sharpe": -2.19, "Corr": 0.49, "Precision": 0.73, "F1": 0.68, "AUC": 0.76, "Confusion Matrix": "[[237, 81], [123, 221]]"},
        {"Model": "Informer", "RMSE": 0.4646, "MAE": 0.3171, "Accuracy (%)": 86.25, "Recall (%)": 75.87, "Sharpe": -4.58, "Corr": 0.91, "Precision": 0.97, "F1": 0.85, "AUC": 0.96, "Confusion Matrix": "[[310, 8], [83, 261]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.0586, "MAE": 0.8068, "Accuracy (%)": 68.88, "Recall (%)": 57.27, "Sharpe": -3.19, "Corr": 0.43, "Precision": 0.77, "F1": 0.66, "AUC": 0.75, "Confusion Matrix": "[[259, 59], [147, 197]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.4185, "MAE": 0.2888, "Accuracy (%)": 88.37, "Recall (%)": 82.85, "Sharpe": -3.40, "Corr": 0.92, "Precision": 0.94, "F1": 0.88, "AUC": 0.96, "Confusion Matrix": "[[300, 18], [59, 285]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.6264, "MAE": 0.4659, "Accuracy (%)": 80.97, "Recall (%)": 70.06, "Sharpe": -5.59, "Corr": 0.82, "Precision": 0.91, "F1": 0.79, "AUC": 0.91, "Confusion Matrix": "[[295, 23], [103, 241]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6121, "MAE": 0.4376, "Accuracy (%)": 80.51, "Recall (%)": 80.23, "Sharpe": -1.62, "Corr": 0.81, "Precision": 0.82, "F1": 0.81, "AUC": 0.89, "Confusion Matrix": "[[257, 61], [68, 276]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.5785, "MAE": 0.3540, "Accuracy (%)": 84.29, "Recall (%)": 71.51, "Sharpe": -5.22, "Corr": 0.88, "Precision": 0.98, "F1": 0.83, "AUC": 0.95, "Confusion Matrix": "[[312, 6], [98, 246]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.5515, "MAE": 0.3853, "Accuracy (%)": 81.12, "Recall (%)": 65.12, "Sharpe": -6.96, "Corr": 0.90, "Precision": 0.98, "F1": 0.78, "AUC": 0.95, "Confusion Matrix": "[[313, 5], [120, 224]]"},
    ],
    "MSFT": [
        {"Model": "Transformer", "RMSE": 0.5499, "MAE": 0.3790, "Accuracy (%)": 85.50, "Recall (%)": 75.95, "Sharpe": -3.88, "Corr": 0.87, "Precision": 0.95, "F1": 0.84, "AUC": 0.95, "Confusion Matrix": "[[307, 14], [82, 259]]"},
        {"Model": "Transformer_no_pe", "RMSE": 0.9405, "MAE": 0.7159, "Accuracy (%)": 68.43, "Recall (%)": 70.38, "Sharpe": -0.77, "Corr": 0.49, "Precision": 0.69, "F1": 0.70, "AUC": 0.74, "Confusion Matrix": "[[213, 108], [101, 240]]"},
        {"Model": "Informer", "RMSE": 0.5900, "MAE": 0.3636, "Accuracy (%)": 87.46, "Recall (%)": 84.16, "Sharpe": -1.27, "Corr": 0.82, "Precision": 0.91, "F1": 0.87, "AUC": 0.94, "Confusion Matrix": "[[292, 29], [54, 287]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.0870, "MAE": 0.8465, "Accuracy (%)": 65.71, "Recall (%)": 55.72, "Sharpe": -5.37, "Corr": 0.40, "Precision": 0.71, "F1": 0.63, "AUC": 0.71, "Confusion Matrix": "[[245, 76], [151, 190]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.6028, "MAE": 0.3704, "Accuracy (%)": 88.07, "Recall (%)": 88.27, "Sharpe": 0.58, "Corr": 0.81, "Precision": 0.89, "F1": 0.88, "AUC": 0.95, "Confusion Matrix": "[[282, 39], [40, 301]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.6637, "MAE": 0.5263, "Accuracy (%)": 77.79, "Recall (%)": 65.10, "Sharpe": -4.54, "Corr": 0.81, "Precision": 0.89, "F1": 0.75, "AUC": 0.90, "Confusion Matrix": "[[293, 28], [119, 222]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6258, "MAE": 0.4839, "Accuracy (%)": 81.12, "Recall (%)": 91.50, "Sharpe": 3.56, "Corr": 0.82, "Precision": 0.76, "F1": 0.83, "AUC": 0.91, "Confusion Matrix": "[[225, 96], [29, 312]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.5106, "MAE": 0.3266, "Accuracy (%)": 88.97, "Recall (%)": 92.96, "Sharpe": 0.79, "Corr": 0.87, "Precision": 0.87, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[272, 49], [24, 317]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.5502, "MAE": 0.3696, "Accuracy (%)": 87.31, "Recall (%)": 90.32, "Sharpe": 0.86, "Corr": 0.86, "Precision": 0.86, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[270, 51], [33, 308]]"},
    ],
    "AMZN": [
        {"Model": "Transformer", "RMSE": 0.4496, "MAE": 0.3190, "Accuracy (%)": 88.37, "Recall (%)": 87.02, "Sharpe": -1.79, "Corr": 0.91, "Precision": 0.90, "F1": 0.88, "AUC": 0.96, "Confusion Matrix": "[[290, 33], [44, 295]]"},
        {"Model": "Transformer_no_pe", "RMSE": 1.1142, "MAE": 0.8202, "Accuracy (%)": 68.73, "Recall (%)": 81.42, "Sharpe": 2.06, "Corr": 0.48, "Precision": 0.66, "F1": 0.73, "AUC": 0.73, "Confusion Matrix": "[[179, 144], [63, 276]]"},
        {"Model": "Informer", "RMSE": 0.4895, "MAE": 0.3387, "Accuracy (%)": 87.46, "Recall (%)": 87.61, "Sharpe": -1.18, "Corr": 0.91, "Precision": 0.88, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[282, 41], [42, 297]]"},
        {"Model": "Informer_no_pe", "RMSE": 0.9978, "MAE": 0.7625, "Accuracy (%)": 65.86, "Recall (%)": 67.55, "Sharpe": -2.68, "Corr": 0.49, "Precision": 0.66, "F1": 0.67, "AUC": 0.70, "Confusion Matrix": "[[207, 116], [110, 229]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.5060, "MAE": 0.3461, "Accuracy (%)": 88.67, "Recall (%)": 91.15, "Sharpe": -0.61, "Corr": 0.90, "Precision": 0.87, "F1": 0.89, "AUC": 0.95, "Confusion Matrix": "[[278, 45], [30, 309]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.5854, "MAE": 0.4350, "Accuracy (%)": 80.82, "Recall (%)": 78.76, "Sharpe": -1.06, "Corr": 0.84, "Precision": 0.83, "F1": 0.81, "AUC": 0.90, "Confusion Matrix": "[[268, 55], [72, 267]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.5716, "MAE": 0.4243, "Accuracy (%)": 82.02, "Recall (%)": 87.02, "Sharpe": 0.96, "Corr": 0.86, "Precision": 0.80, "F1": 0.83, "AUC": 0.91, "Confusion Matrix": "[[248, 75], [44, 295]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.4593, "MAE": 0.3238, "Accuracy (%)": 88.82, "Recall (%)": 93.22, "Sharpe": 0.89, "Corr": 0.91, "Precision": 0.86, "F1": 0.90, "AUC": 0.96, "Confusion Matrix": "[[272, 51], [23, 316]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.4939, "MAE": 0.3503, "Accuracy (%)": 88.22, "Recall (%)": 93.81, "Sharpe": 0.74, "Corr": 0.91, "Precision": 0.85, "F1": 0.89, "AUC": 0.95, "Confusion Matrix": "[[266, 57], [21, 318]]"},
    ],
    "GOOG": [
        {"Model": "Transformer", "RMSE": 0.5648, "MAE": 0.4000, "Accuracy (%)": 84.44, "Recall (%)": 82.34, "Sharpe": -1.35, "Corr": 0.88, "Precision": 0.86, "F1": 0.84, "AUC": 0.93, "Confusion Matrix": "[[284, 44], [59, 275]]"},
        {"Model": "Transformer_no_pe", "RMSE": 1.1464, "MAE": 0.8612, "Accuracy (%)": 62.39, "Recall (%)": 55.39, "Sharpe": -2.22, "Corr": 0.43, "Precision": 0.65, "F1": 0.60, "AUC": 0.68, "Confusion Matrix": "[[228, 100], [149, 185]]"},
        {"Model": "Informer", "RMSE": 0.6075, "MAE": 0.4390, "Accuracy (%)": 87.16, "Recall (%)": 88.92, "Sharpe": 0.41, "Corr": 0.90, "Precision": 0.86, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[280, 48], [37, 297]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.0838, "MAE": 0.8129, "Accuracy (%)": 62.08, "Recall (%)": 48.20, "Sharpe": -4.68, "Corr": 0.47, "Precision": 0.67, "F1": 0.56, "AUC": 0.72, "Confusion Matrix": "[[250, 78], [173, 161]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.5908, "MAE": 0.4139, "Accuracy (%)": 87.01, "Recall (%)": 80.54, "Sharpe": -2.78, "Corr": 0.90, "Precision": 0.93, "F1": 0.86, "AUC": 0.96, "Confusion Matrix": "[[307, 21], [65, 269]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.8140, "MAE": 0.5963, "Accuracy (%)": 76.59, "Recall (%)": 58.98, "Sharpe": -6.29, "Corr": 0.79, "Precision": 0.92, "F1": 0.72, "AUC": 0.88, "Confusion Matrix": "[[310, 18], [137, 197]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6666, "MAE": 0.5110, "Accuracy (%)": 80.51, "Recall (%)": 80.24, "Sharpe": -0.04, "Corr": 0.84, "Precision": 0.81, "F1": 0.81, "AUC": 0.90, "Confusion Matrix": "[[265, 63], [66, 268]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.5283, "MAE": 0.3672, "Accuracy (%)": 86.86, "Recall (%)": 83.53, "Sharpe": -1.69, "Corr": 0.90, "Precision": 0.90, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[296, 32], [55, 279]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.5992, "MAE": 0.4338, "Accuracy (%)": 85.35, "Recall (%)": 77.25, "Sharpe": -1.96, "Corr": 0.89, "Precision": 0.92, "F1": 0.84, "AUC": 0.95, "Confusion Matrix": "[[307, 21], [76, 258]]"},
    ],
    "META": [
        {"Model": "Transformer", "RMSE": 0.5021, "MAE": 0.3245, "Accuracy (%)": 88.80, "Recall (%)": 88.77, "Sharpe": 0.69, "Corr": 0.90, "Precision": 0.92, "F1": 0.90, "AUC": 0.94, "Confusion Matrix": "[[239, 30], [42, 332]]"},
        {"Model": "Transformer_no_pe", "RMSE": 1.0199, "MAE": 0.7134, "Accuracy (%)": 69.52, "Recall (%)": 63.37, "Sharpe": -1.08, "Corr": 0.48, "Precision": 0.80, "F1": 0.71, "AUC": 0.76, "Confusion Matrix": "[[210, 59], [137, 237]]"},
        {"Model": "Informer", "RMSE": 0.5140, "MAE": 0.3666, "Accuracy (%)": 87.09, "Recall (%)": 82.35, "Sharpe": -0.84, "Corr": 0.92, "Precision": 0.95, "F1": 0.88, "AUC": 0.95, "Confusion Matrix": "[[252, 17], [66, 308]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.1087, "MAE": 0.7953, "Accuracy (%)": 59.10, "Recall (%)": 54.28, "Sharpe": -0.72, "Corr": 0.33, "Precision": 0.69, "F1": 0.61, "AUC": 0.65, "Confusion Matrix": "[[177, 92], [171, 203]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.4432, "MAE": 0.3190, "Accuracy (%)": 88.02, "Recall (%)": 86.63, "Sharpe": 0.94, "Corr": 0.93, "Precision": 0.92, "F1": 0.89, "AUC": 0.95, "Confusion Matrix": "[[242, 27], [50, 324]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.6293, "MAE": 0.4457, "Accuracy (%)": 81.34, "Recall (%)": 75.40, "Sharpe": -1.41, "Corr": 0.86, "Precision": 0.91, "F1": 0.82, "AUC": 0.91, "Confusion Matrix": "[[241, 28], [92, 282]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6452, "MAE": 0.4990, "Accuracy (%)": 75.74, "Recall (%)": 62.83, "Sharpe": -4.07, "Corr": 0.88, "Precision": 0.93, "F1": 0.75, "AUC": 0.91, "Confusion Matrix": "[[252, 17], [139, 235]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.4439, "MAE": 0.3027, "Accuracy (%)": 89.58, "Recall (%)": 93.58, "Sharpe": 3.18, "Corr": 0.93, "Precision": 0.89, "F1": 0.91, "AUC": 0.96, "Confusion Matrix": "[[226, 43], [24, 350]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.4599, "MAE": 0.3176, "Accuracy (%)": 89.11, "Recall (%)": 90.37, "Sharpe": 2.38, "Corr": 0.94, "Precision": 0.91, "F1": 0.91, "AUC": 0.96, "Confusion Matrix": "[[235, 34], [36, 338]]"},
    ],
    "NVDA": [
        {"Model": "Transformer", "RMSE": 0.5636, "MAE": 0.4279, "Accuracy (%)": 87.46, "Recall (%)": 84.66, "Sharpe": -0.94, "Corr": 0.90, "Precision": 0.92, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[270, 27], [56, 309]]"},
        {"Model": "Transformer_no_pe", "RMSE": 1.1898, "MAE": 0.9589, "Accuracy (%)": 62.24, "Recall (%)": 68.49, "Sharpe": 2.22, "Corr": 0.38, "Precision": 0.65, "F1": 0.67, "AUC": 0.68, "Confusion Matrix": "[[162, 135], [115, 250]]"},
        {"Model": "Informer", "RMSE": 0.5646, "MAE": 0.4152, "Accuracy (%)": 85.20, "Recall (%)": 81.10, "Sharpe": -0.45, "Corr": 0.89, "Precision": 0.91, "F1": 0.86, "AUC": 0.94, "Confusion Matrix": "[[268, 29], [69, 296]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.4298, "MAE": 1.0872, "Accuracy (%)": 64.95, "Recall (%)": 69.59, "Sharpe": -0.90, "Corr": 0.38, "Precision": 0.68, "F1": 0.69, "AUC": 0.69, "Confusion Matrix": "[[176, 121], [111, 254]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.5152, "MAE": 0.3758, "Accuracy (%)": 86.71, "Recall (%)": 90.14, "Sharpe": 2.72, "Corr": 0.90, "Precision": 0.86, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[245, 52], [36, 329]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.6546, "MAE": 0.5042, "Accuracy (%)": 83.69, "Recall (%)": 83.56, "Sharpe": 0.63, "Corr": 0.83, "Precision": 0.86, "F1": 0.85, "AUC": 0.91, "Confusion Matrix": "[[249, 48], [60, 305]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6780, "MAE": 0.5255, "Accuracy (%)": 81.42, "Recall (%)": 79.73, "Sharpe": 0.21, "Corr": 0.82, "Precision": 0.86, "F1": 0.83, "AUC": 0.90, "Confusion Matrix": "[[248, 49], [74, 291]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.5189, "MAE": 0.3881, "Accuracy (%)": 84.59, "Recall (%)": 80.55, "Sharpe": -0.43, "Corr": 0.90, "Precision": 0.90, "F1": 0.85, "AUC": 0.94, "Confusion Matrix": "[[266, 31], [71, 294]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.5446, "MAE": 0.4143, "Accuracy (%)": 86.56, "Recall (%)": 89.59, "Sharpe": 1.51, "Corr": 0.91, "Precision": 0.87, "F1": 0.88, "AUC": 0.94, "Confusion Matrix": "[[246, 51], [38, 327]]"},
    ],
    "TSLA": [
        {"Model": "Transformer", "RMSE": 0.5020, "MAE": 0.3946, "Accuracy (%)": 86.25, "Recall (%)": 80.00, "Sharpe": -1.80, "Corr": 0.91, "Precision": 0.89, "F1": 0.84, "AUC": 0.94, "Confusion Matrix": "[[331, 31], [60, 240]]"},
        {"Model": "Transformer_no_pe", "RMSE": 1.0492, "MAE": 0.8135, "Accuracy (%)": 69.64, "Recall (%)": 63.33, "Sharpe": -2.89, "Corr": 0.57, "Precision": 0.68, "F1": 0.65, "AUC": 0.77, "Confusion Matrix": "[[271, 91], [110, 190]]"},
        {"Model": "Informer", "RMSE": 0.4568, "MAE": 0.3513, "Accuracy (%)": 88.07, "Recall (%)": 85.67, "Sharpe": -1.39, "Corr": 0.93, "Precision": 0.88, "F1": 0.87, "AUC": 0.95, "Confusion Matrix": "[[326, 36], [43, 257]]"},
        {"Model": "Informer_no_pe", "RMSE": 1.1764, "MAE": 0.9404, "Accuracy (%)": 66.31, "Recall (%)": 65.67, "Sharpe": 3.42, "Corr": 0.48, "Precision": 0.62, "F1": 0.64, "AUC": 0.74, "Confusion Matrix": "[[242, 120], [103, 197]]"},
        {"Model": "Informer_no_probs", "RMSE": 0.4218, "MAE": 0.3211, "Accuracy (%)": 87.92, "Recall (%)": 82.67, "Sharpe": -3.12, "Corr": 0.93, "Precision": 0.90, "F1": 0.86, "AUC": 0.96, "Confusion Matrix": "[[334, 28], [52, 248]]"},
        {"Model": "Transformer_no_imf", "RMSE": 0.5867, "MAE": 0.4492, "Accuracy (%)": 81.72, "Recall (%)": 88.00, "Sharpe": 1.20, "Corr": 0.86, "Precision": 0.76, "F1": 0.81, "AUC": 0.92, "Confusion Matrix": "[[277, 85], [36, 264]]"},
        {"Model": "Informer_no_imf", "RMSE": 0.6504, "MAE": 0.4917, "Accuracy (%)": 81.27, "Recall (%)": 76.67, "Sharpe": -2.03, "Corr": 0.84, "Precision": 0.81, "F1": 0.79, "AUC": 0.90, "Confusion Matrix": "[[308, 54], [70, 230]]"},
        {"Model": "Transformer_no_tech", "RMSE": 0.4039, "MAE": 0.3030, "Accuracy (%)": 88.37, "Recall (%)": 82.67, "Sharpe": -2.44, "Corr": 0.93, "Precision": 0.91, "F1": 0.87, "AUC": 0.96, "Confusion Matrix": "[[337, 25], [52, 248]]"},
        {"Model": "Informer_no_tech", "RMSE": 0.4671, "MAE": 0.3528, "Accuracy (%)": 89.27, "Recall (%)": 89.67, "Sharpe": -0.83, "Corr": 0.94, "Precision": 0.87, "F1": 0.88, "AUC": 0.96, "Confusion Matrix": "[[322, 40], [31, 269]]"},
    ]
}

# CSV headers
headers = ["Model", "RMSE", "MAE", "Accuracy (%)", "Recall (%)", "Sharpe", "Corr", "Precision", "F1", "AUC", "Confusion Matrix"]

# Generate a CSV file for each stock
for stock, data in stocks_data.items():
    filename = f"{stock}_performance.csv"
    with open(filename, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

print("CSV files generated for each stock: AAPL, MSFT, AMZN, GOOG, META, NVDA, TSLA")

CSV files generated for each stock: AAPL, MSFT, AMZN, GOOG, META, NVDA, TSLA
